## Import Libraries and Set Up the Environment


In [37]:
import numpy as np
import pandas as pd, csv 
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense


## Importing Dataset

In [38]:
dataset = pd.read_csv('medical_data.csv')
len(dataset)

407

In [39]:
import re
import nltk

nltk.data.path.append('/Users/bahodirnematjonov/Desktop/AI-Machine-Learning-Deep-Learning-Computer-Vision/nltk_data')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []
ps = PorterStemmer()

for j in range (0, 407):
    reviews = re.sub('[^a-zA-Z]', ' ', dataset['Patient_Problem'][j])
    reviews = reviews.lower()
    reviews = reviews.split()
    all_words = stopwords.words('english')
    reviews = [ ps.stem(word) for word in reviews if word not in all_words]
    result = ' '.join(reviews)
    corpus.append(result) 
'''
disease = []

for j in range (0, 407):
    reviews = re.sub('[^a-zA-Z]', ' ', dataset['Disease'][j])
    reviews = reviews.lower()
    reviews = reviews.split()
    all_words = stopwords.words('english')
    reviews = [ ps.stem(word) for word in reviews if word not in all_words]
    result = ' '.join(reviews)
    disease.append(result) 


prescription = []

for j in range (0, 407):
    reviews = re.sub('[^a-zA-Z]', ' ', dataset['Prescription'][j])
    reviews = reviews.lower()
    reviews = reviews.split()
    all_words = stopwords.words('english')
    reviews = [ ps.stem(word) for word in reviews if word not in all_words]
    result = ' '.join(reviews)
    prescription.append(result) '''



        




"\ndisease = []\n\nfor j in range (0, 407):\n    reviews = re.sub('[^a-zA-Z]', ' ', dataset['Disease'][j])\n    reviews = reviews.lower()\n    reviews = reviews.split()\n    all_words = stopwords.words('english')\n    reviews = [ ps.stem(word) for word in reviews if word not in all_words]\n    result = ' '.join(reviews)\n    disease.append(result) \n\n\nprescription = []\n\nfor j in range (0, 407):\n    reviews = re.sub('[^a-zA-Z]', ' ', dataset['Prescription'][j])\n    reviews = reviews.lower()\n    reviews = reviews.split()\n    all_words = stopwords.words('english')\n    reviews = [ ps.stem(word) for word in reviews if word not in all_words]\n    result = ' '.join(reviews)\n    prescription.append(result) "

In [40]:
corpus

(['constant fatigu muscl weak struggl stay awak',
  'frequent sever migrain sensit light sound',
  'sudden weight gain feel cold especi hand feet',
  'high fever sore throat swollen lymph node feel weak',
  'excess thirst frequent urin dri mouth persist',
  'sharp stab chest pain worsen lie inhal deepli',
  'unexplain weight loss night sweat feel feverish',
  'sever abdomin pain radiat back nausea without vomit',
  'joint swell stiff morn especi finger wrist',
  'recurr pain blister around mouth nose',
  'progress difficulti swallow accompani heartburn',
  'chronic diarrhea abdomin cramp ga',
  'intens itch especi night visibl burrow skin',
  'yellow skin eye dark urin gener fatigu',
  'sudden sever headach known caus blur vision',
  'persist cough blood ting sputum night sweat',
  'frequent noseble easi bruis prolong bleed cut',
  'sever itch rash consum certain food',
  'repeat urinari tract infect burn sensat urin',
  'difficulti concentr memori laps confus',
  'feel extrem fatigu f

## Label Encoding

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()


In [43]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
y

array([['Chronic Fatigue Syndrome',
        'Cognitive behavioral therapy, graded exercise therapy.'],
       ['Migraine with Aura',
        'Prescription triptans, avoid triggers like bright lights.'],
       ['Hypothyroidism',
        'Levothyroxine to regulate thyroid hormone levels.'],
       ['Mononucleosis', 'Rest and hydration, ibuprofen for pain.'],
       ['Diabetes Mellitus', 'Insulin therapy and lifestyle changes.'],
       ['Pericarditis', 'Anti-inflammatory medication, rest.'],
       ['Tuberculosis', 'Antibiotics, rest, and a high-calorie diet.'],
       ['Pancreatitis', 'IV fluids, pain relief, and sometimes surgery.'],
       ['Rheumatoid Arthritis', 'DMARDs and NSAIDs, physical therapy.'],
       ['Herpes Simplex', 'Antiviral medication, pain relief creams.'],
       ['Esophageal Reflux', 'Antacids, avoid spicy and fatty foods.'],
       ['Irritable Bowel Syndrome',
        'Fiber supplements, antispasmodic drugs.'],
       ['Scabies', 'Permethrin cream, wash all cloth

In [48]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
disease_encoder = lb.fit_transform(dataset['Disease'])
prescription_encoder = lb.fit_transform(dataset['Prescription'])


In [50]:
disease_encoder

array([ 31, 114,  88, 116,  48, 134, 167, 127, 148,  84,  60,  94, 149,
        80, 159, 103,  79,  65, 173,  54,  31, 112,  15, 128, 168, 148,
        95,  42,  90, 107, 121,  50,  76,  34,  22,  85,  80, 137, 136,
       144, 113,  73,  39,  30, 132, 120,  31, 150,  48,  91, 157, 107,
       148,  87, 165,  57, 138,  96,  61, 136,  31, 148,  15, 130, 112,
        88, 156, 120,  89,  69,  85,  28,  53, 136,  83, 173,  30,   5,
       107, 129,  31, 115,  15,  71,   3, 150, 148,  25,  23, 122, 168,
       173, 143, 136, 177,  69,  88, 130,  90, 129, 112,  94,  39,  24,
        98, 120,  31, 150, 169,  90,  87,  85,  88,  18,  19, 148, 136,
       118,  82, 137, 112,  69,  30, 150,  11, 148,  46, 120,  15,  87,
        39, 141,  90,  44, 137, 173,  26, 125,  89,  79,  13,  80, 139,
       165,  62,  75,  31, 118,  18,  55,   4,  88,  56,  23,  88, 136,
        43, 167,  19, 117, 162,  94,  39,  24, 132, 146,  93, 150, 169,
        90, 151,  27,  17, 125, 165,  87,   4,  54,   9,  10,  3

In [52]:
prescription_encoder

array([122, 307, 221, 325, 206,  18,  34, 187, 143,  75,  13, 173, 282,
        71, 190, 113, 324,  89,  33, 124, 318,  85, 199, 151, 208, 149,
       152, 365, 177,  48, 135, 224,  80, 228,  84,  61,  74,  30, 100,
        45, 345, 184, 186, 203, 292, 305,  91, 281, 237, 334,   6, 330,
       253, 222, 259, 183, 254, 227, 266, 381,   5, 302, 197, 216, 308,
       220,  28, 301, 264, 309,  59,  23,  78, 380, 279,  22, 229, 123,
       329,  93, 180,  86, 199,  12,  54, 287, 144,  29, 246, 377, 204,
        25, 362, 379, 169, 310, 219, 215, 332, 126, 299, 316,   4, 200,
       293, 304, 167, 289, 209, 333,  67,  63, 358,  95, 343, 147, 134,
       290, 245,  31, 303,   9, 193, 294, 119, 253, 104, 137, 202, 357,
       363, 298, 336,  46, 384,  35, 328, 192,  65,  90, 350,  76,  40,
       260, 312, 263, 268, 291,  19, 244, 364, 360, 164, 248, 356, 273,
        42,  69,  92, 159,  83, 175,  81, 375, 295, 331, 211, 276, 210,
         3,  55, 387, 243, 192,  82,  68, 239, 120, 258, 261, 10